In [6]:
# Text text processing library
import torchtext
from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import itertools as it
from models import *
from helpers import *
# import main
import matplotlib.pyplot as plt
import spacy
import time
import itertools as it
MAX_LEN = 20
MIN_FREQ = 5

In [2]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

BOS_WORD = '<s>'
EOS_WORD = '</s>'
DE = data.Field(tokenize=tokenize_de)

# only target needs BOS/EOS:
EN = data.Field(tokenize=tokenize_en, init_token = BOS_WORD, eos_token = EOS_WORD) 

train, val, test = datasets.IWSLT.splits(exts=('.de', '.en'), fields=(DE, EN), 
                                         filter_pred=lambda x: len(vars(x)['src']) <= MAX_LEN and 
                                         len(vars(x)['trg']) <= MAX_LEN)

In [3]:
DE.build_vocab(train.src, min_freq=MIN_FREQ)
EN.build_vocab(train.trg, min_freq=MIN_FREQ)

pred_set = []
for i, line in enumerate(open("source_test.txt"), 1):
    words = line.split()[:-1]
    pred_set.append([DE.vocab.stoi[s] for s in words])

train_iter, val_iter, test_iter = data.BucketIterator.splits((train, val, test), batch_size=32, device=-1,
                                                  repeat=False, sort_key=lambda x: len(x.src))

In [17]:
batch = next(iter(test_iter))
# sent_inspect(batch,4)
def sent_inspect(batch, idx=0):
    print("Source")
    print(' '.join([DE.vocab.itos[w] for w in batch.src.data[:,idx]]))
    print("Target")
    print(' '.join([EN.vocab.itos[w] for w in batch.trg.data[:,idx]]))
# print(DE.vocab.stoi['<pad>'])

In [81]:
for i in range(10):
    print(' '.join(DE.vocab.itos[w] for w in debug_set[i]))
    print(' '.join(EN.vocab.itos[w] for w in debug_ans[i]))    

wo sich der Lauf des <unk> früher befand .
<s> We started to be able to see where the <unk> used to flow . </s> <pad> <pad>
Das führte mich dazu , Satellitenbilder zu benutzen .
<s> This is really what brought me to using satellite imagery . </s> <pad> <pad> <pad> <pad> <pad>
Wie unterscheidet sich diese Geschichte von der anderen ?
<s> How is this story different ? </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Im Raum war ein junger Mann , <unk> .
<s> One of the young men in the room was <unk> . </s> <pad> <pad> <pad> <pad> <pad>
Ich brachte rund 90 junge <unk> Führungskräfte zusammen .
<s> I brought together about 90 young Somali leaders . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Es vergeht vielleicht ein Jahr , aber nichts .
<s> Maybe a year passes , nothing . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Sein Dorf liegt in der Nähe von <unk> .
<s> His village is near <unk> . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Ich d

In [6]:
save_checkpoint(bs_encoder, at_decoder, filename='saved_models/attn_big_0.epoch_10.ckpt.tar')
evaluator.predict(pred_set, fn='predictions_real_300_eos-2.txt',beam_size=300, ignore_eos=True)
# evaluator.predict(pred_set, fn='predictions_real_200_eos.txt',beam_size=200, ignore_eos=True)

torch.Size([3468000])
torch.Size([3468000])
Writing predictions to predictions_real_300_eos-2.txt...
Computing predictions took 325.554955 seconds


In [79]:
train_iter, val_iter = data.BucketIterator.splits((train, val), batch_size=32, device=-1,
                                                  repeat=False, sort_key=lambda x: len(x.src))
bs_encoder = BaseEncoder(DE, hidden_size=500, num_layers=4, word_features=500)
bs_decoder = BaseDecoder(EN, hidden_size=500, num_layers=4, word_features=500)
trainer = NMTTrainer([bs_encoder, bs_decoder], DE, EN, lrn_rate=0.7, 
                     lrn_decay='adaptive', reverse_enc_input=False)
evaluator = NMTEvaluator([bs_encoder, bs_decoder], DE, EN, reverse_enc_input=False)
trainer.train(train_iter, verbose=True, le=evaluator, val_iter=val_iter)

Target padding token: 1
Using CUDA...
Target padding token: 1
Using CUDA...
Epoch 0, loss: 75.106003, norm: 5.068851, elapsed: 601.591950, lrn_rate: 0.700000
Validation time: 1.177268 seconds
Validation set metric: 15.046797
Epoch 1, loss: 64.709015, norm: 6.823921, elapsed: 1202.462563, lrn_rate: 0.700000
Validation time: 1.123111 seconds
Validation set metric: 10.318951
Epoch 2, loss: 55.162987, norm: 5.810015, elapsed: 1815.646121, lrn_rate: 0.700000
Validation time: 1.158648 seconds
Validation set metric: 8.200983
Epoch 3, loss: 48.013638, norm: 6.298200, elapsed: 2430.358610, lrn_rate: 0.700000
Validation time: 1.222892 seconds
Validation set metric: 7.256750
Epoch 4, loss: 55.855797, norm: 8.171973, elapsed: 3034.166331, lrn_rate: 0.700000
Validation time: 1.161787 seconds
Validation set metric: 7.055706
Epoch 5, loss: 51.494362, norm: 9.309718, elapsed: 3637.575163, lrn_rate: 0.700000
Validation time: 1.184436 seconds
Validation set metric: 7.108363
Decaying LR to 0.350000
Epoch

KeyboardInterrupt: 

In [4]:
train_iter, val_iter = data.BucketIterator.splits((train, val), batch_size=32, device=-1,
                                                  repeat=False, sort_key=lambda x: len(x.src))
bs_encoder = BaseEncoder(DE, hidden_size=300, num_layers=4, word_features=300, 
                         dropout=0.2)
# TODO: decide whether to add dropout to output states of encoder!
at_decoder = AttnDecoder(EN, hidden_size=300, num_layers=4, word_features=300, dropout=0.2,
                         tie_weights=True, enc_linear=300)
trainer = NMTTrainer([bs_encoder, at_decoder], DE, EN, lrn_rate=1.0, 
                     lrn_decay='adaptive', attention=True,
                     clip_norm=5, lrn_decay_force=8)
evaluator = NMTEvaluator([bs_encoder, at_decoder], DE, EN, attention=True,
                        record_attention=False)
trainer.train(train_iter, verbose=True, le=evaluator, val_iter=val_iter,
              save_model_fn='attn_med_3')

Target padding token: 1
Using CUDA...
Target padding token: 1
Using CUDA...
Innitializing parameters status:  True
Epoch 0, loss: 76.252197, norm: 4.725919, elapsed: 353.864778, lrn_rate: 1.000000
Validation time: 0.583281 seconds
Validation set metric: 16.969514
Saving model to saved_models/attn_med_3.epoch_0.ckpt.tar
Epoch 1, loss: 76.925499, norm: 5.523378, elapsed: 713.133499, lrn_rate: 1.000000
Validation time: 0.572580 seconds
Validation set metric: 12.591491
Saving model to saved_models/attn_med_3.epoch_1.ckpt.tar
Epoch 2, loss: 55.661785, norm: 5.266655, elapsed: 1077.052984, lrn_rate: 1.000000
Validation time: 0.495382 seconds
Validation set metric: 8.166244
Saving model to saved_models/attn_med_3.epoch_2.ckpt.tar
Epoch 3, loss: 51.431374, norm: 7.361608, elapsed: 1439.067216, lrn_rate: 1.000000
Validation time: 0.569628 seconds
Validation set metric: 7.040756
Saving model to saved_models/attn_med_3.epoch_3.ckpt.tar
Epoch 4, loss: 52.481609, norm: 5.975768, elapsed: 1800.62142

KeyboardInterrupt: 

In [ ]:
cnt = 0
for hs, tw, force, drop in it.product([200,400,600],[False, True],
                                      [4,8],[0.2, 0.4]):
    print(hs, tw, force, drop)
    train_iter, val_iter = data.BucketIterator.splits((train, val), batch_size=32, device=-1,
                                                      repeat=False, sort_key=lambda x: len(x.src))
    bs_encoder = BaseEncoder(DE, hidden_size=hs, num_layers=4, word_features=hs, 
                             dropout=drop)
    # TODO: decide whether to add dropout to output states of encoder!
    at_decoder = AttnDecoder(EN, hidden_size=hs, num_layers=4, word_features=hs, dropout=drop,
                             tie_weights=tw)
    trainer = NMTTrainer([bs_encoder, at_decoder], DE, EN, lrn_rate=1.0, 
                         lrn_decay='adaptive', attention=True,
                         clip_norm=5, lrn_decay_force=force)
    evaluator = NMTEvaluator([bs_encoder, at_decoder], DE, EN, attention=True,
                            record_attention=False)
    trainer.train(train_iter, verbose=True, le=evaluator, val_iter=val_iter,
                  save_model_fn='attn_grid_%d' % cnt, num_iter=10)
    cnt += 1

200 False 4 0.2
Target padding token: 1
Using CUDA...
Target padding token: 1
Using CUDA...
Innitializing parameters status:  True
Epoch 0, loss: 83.561142, norm: 4.071474, elapsed: 271.279075, lrn_rate: 1.000000
Validation time: 0.360532 seconds
Validation set metric: 22.966567
Saving model to saved_models/attn_grid_0.epoch_0.ckpt.tar
Epoch 1, loss: 82.306152, norm: 4.954175, elapsed: 548.610403, lrn_rate: 1.000000
Validation time: 0.354557 seconds
Validation set metric: 15.265598
Saving model to saved_models/attn_grid_0.epoch_1.ckpt.tar
Epoch 2, loss: 59.941078, norm: 4.696200, elapsed: 828.961941, lrn_rate: 1.000000
Validation time: 0.392126 seconds
Validation set metric: 9.756422
Saving model to saved_models/attn_grid_0.epoch_2.ckpt.tar
Epoch 3, loss: 55.912140, norm: 5.782441, elapsed: 1111.078448, lrn_rate: 1.000000
Validation time: 0.360725 seconds
Validation set metric: 7.972600
Saving model to saved_models/attn_grid_0.epoch_3.ckpt.tar
Decaying LR to 0.500000
Epoch 4, loss: 54.

Epoch 5, loss: 52.777771, norm: 5.691047, elapsed: 1657.517624, lrn_rate: 1.000000
Validation time: 0.355772 seconds
Validation set metric: 7.938821
Saving model to saved_models/attn_grid_3.epoch_5.ckpt.tar
Epoch 6, loss: 61.794411, norm: 6.426108, elapsed: 1936.887601, lrn_rate: 1.000000
Validation time: 0.348664 seconds
Validation set metric: 7.579755
Saving model to saved_models/attn_grid_3.epoch_6.ckpt.tar
Epoch 7, loss: 58.316906, norm: 5.636143, elapsed: 2217.693143, lrn_rate: 1.000000
Validation time: 0.371190 seconds
Validation set metric: 7.299893
Saving model to saved_models/attn_grid_3.epoch_7.ckpt.tar
Decaying LR to 0.500000
Epoch 8, loss: 51.196472, norm: 5.013029, elapsed: 2497.512262, lrn_rate: 0.500000
Validation time: 0.349605 seconds
Validation set metric: 6.763396
Saving model to saved_models/attn_grid_3.epoch_8.ckpt.tar
Decaying LR to 0.250000
Epoch 9, loss: 47.804970, norm: 5.102167, elapsed: 2778.161504, lrn_rate: 0.250000
Validation time: 0.367842 seconds
Validat

Epoch 0, loss: 80.696709, norm: 4.487724, elapsed: 268.828376, lrn_rate: 1.000000
Validation time: 0.356540 seconds
Validation set metric: 19.476453
Saving model to saved_models/attn_grid_7.epoch_0.ckpt.tar
Epoch 1, loss: 84.230835, norm: 5.347474, elapsed: 540.498028, lrn_rate: 1.000000
Validation time: 0.353929 seconds
Validation set metric: 14.954864
Saving model to saved_models/attn_grid_7.epoch_1.ckpt.tar
Epoch 2, loss: 64.211594, norm: 4.860816, elapsed: 813.037319, lrn_rate: 1.000000
Validation time: 0.354883 seconds
Validation set metric: 11.287956
Saving model to saved_models/attn_grid_7.epoch_2.ckpt.tar
Epoch 3, loss: 60.588867, norm: 6.156037, elapsed: 1087.799996, lrn_rate: 1.000000
Validation time: 0.348687 seconds
Validation set metric: 9.001583
Saving model to saved_models/attn_grid_7.epoch_3.ckpt.tar
Epoch 4, loss: 62.230396, norm: 5.461389, elapsed: 1363.474364, lrn_rate: 1.000000
Validation time: 0.371995 seconds
Validation set metric: 8.228699
Saving model to saved_m

In [5]:
# Load a model and continue training:
ld_enc, ld_dec = load_checkpoint('saved_models/attn_med_1.epoch_8.ckpt.tar')
ld_encoder = BaseEncoder(DE, hidden_size=400, num_layers=4, word_features=400, 
                         dropout=0.2)
ld_decoder = AttnDecoder(EN, hidden_size=400, num_layers=4, word_features=400, 
                         dropout=0.2)
set_parameters(ld_encoder, ld_enc)
set_parameters(ld_decoder, ld_dec)
evaluator = NMTEvaluator([ld_encoder, ld_decoder], DE, EN, attention=True,
                        record_attention=False)
print(evaluator.evaluate(val_iter))
trainer = NMTTrainer([ld_encoder, ld_decoder], DE, EN, lrn_rate=0.5, 
                     lrn_decay='adaptive', attention=True,
                     clip_norm=5, lrn_decay_force=2)
trainer.train(train_iter, verbose=True, le=evaluator, val_iter=val_iter,
              save_model_fn='attn_med_1b', init_parameters=False)

Target padding token: 1
Using CUDA...
Validation time: 0.677992 seconds
5.322171705651731
Target padding token: 1
Using CUDA...
Innitializing parameters status:  False
Epoch 0, loss: 39.508476, norm: 6.285943, elapsed: 426.947739, lrn_rate: 0.500000
Validation time: 0.728775 seconds
Validation set metric: 5.209565
Saving model to saved_models/attn_med_1b.epoch_0.ckpt.tar
Epoch 1, loss: 44.648376, norm: 7.371956, elapsed: 859.301896, lrn_rate: 0.500000
Validation time: 0.614820 seconds
Validation set metric: 5.190459
Saving model to saved_models/attn_med_1b.epoch_1.ckpt.tar
Decaying LR to 0.250000
Epoch 2, loss: 38.062698, norm: 7.248244, elapsed: 1291.240771, lrn_rate: 0.250000
Validation time: 0.749838 seconds
Validation set metric: 5.085312
Saving model to saved_models/attn_med_1b.epoch_2.ckpt.tar
Decaying LR to 0.125000
Epoch 3, loss: 32.695580, norm: 6.297701, elapsed: 1722.055861, lrn_rate: 0.125000
Validation time: 0.788171 seconds
Validation set metric: 5.076100
Saving model to 

KeyboardInterrupt: 

In [6]:
print(evaluator.evaluate(val_iter))
print(evaluator.evaluate(test_iter))

Validation time: 0.659822 seconds
5.120132318658038
Validation time: 0.872873 seconds
6.3874894806368365


In [39]:
print(evaluator.evaluate(val_iter))
print(len(train_iter))
print(evaluator.attns_log[0][4])
val_iter.init_epoch()
batch = next(iter(val_iter))
sent_inspect(batch, 4)

Validation time: 0.456792 seconds
6.048956472651303
3722
Variable containing:
 0.6505  0.1110  0.0659  0.1725
 0.5297  0.1554  0.2806  0.0343
 0.0089  0.4239  0.4734  0.0938
 0.0180  0.0371  0.9122  0.0327
 0.0822  0.0702  0.3141  0.5335
 0.4222  0.2591  0.0997  0.2190
 0.1612  0.3041  0.4915  0.0432
 0.2962  0.4637  0.0781  0.1620
[torch.cuda.FloatTensor of size 8x4 (GPU 0)]

Source
Ich hatte Angst –
Target
<s> And I was scared . </s> <pad> <pad>


In [6]:
# save_checkpoint(bs_encoder, at_decoder, filename='saved_models/attn_big_0.epoch_10.ckpt.tar')
ld_enc, ld_dec = load_checkpoint('saved_models/attn_med_2.epoch_12.ckpt.tar')
ld_encoder = BaseEncoder(DE, hidden_size=500, num_layers=4, word_features=500, dropout=0.3)
ld_decoder = AttnDecoder(EN, hidden_size=500, num_layers=4, word_features=500, dropout=0.3)
set_parameters(ld_encoder, ld_enc)
set_parameters(ld_decoder, ld_dec)
evaluator = NMTEvaluator([ld_encoder, ld_decoder], DE, EN, attention=True,
                        record_attention=False)
print(evaluator.evaluate(val_iter))
print(evaluator.evaluate(test_iter))
evaluator.predict(pred_set, fn='predictions_med_2_200.txt',beam_size=200, ignore_eos=True)

Target padding token: 1
Using CUDA...
Validation time: 0.834653 seconds
4.805516453880054
Validation time: 1.456081 seconds
5.948858129956995
torch.Size([2312000])
torch.Size([2312000])
Writing predictions to predictions_med_2_200.txt...
Computing predictions took 145.327281 seconds


In [11]:
test_iter.init_epoch()
debug_iter = iter(test_iter)
for i in range(10):
    batch = next(debug_iter)
debug_set = [batch.src.data[:, i] for i in range(batch.src.data.size(1))]
debug_ans = [batch.trg.data[:, i] for i in range(batch.trg.data.size(1))]
evaluator.predict(pred_set, fn='predictions_real.txt',beam_size=200)

In [10]:
a = torch.Tensor([-np.inf])
print(a)


-inf
[torch.FloatTensor of size 1]



In [6]:
print(3 < np.inf)

True


## PROBABLY NOT RELEVANT STUFF BELOW HERE

In [ ]:
bs_decoder_new = AttnDecoder(EN, hidden_size=500, num_layers=4, word_features=500, dropout=0.2)
old_params = list(bs_decoder.parameters())
for i,p in enumerate(bs_decoder_new.parameters()):
    p.data = old_params[i].data
print(list(bs_decoder_new.parameters())[0])
print(list(bs_decoder.parameters())[0])
bs_decoder_new.lstm.flatten_parameters()